In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 

最上面抽取特征这一部分学习了github上学长的代码，不过学长的代码也存在一定的问题。
包括：
1. 文章分类时循环的次数
2. 对频数统计的理解
3. MLE计算phi时使用的分母

后面预测部分都是我自己完成的

In [2]:
# 读取联邦党人文集文件
papers = open('The-Federalist-Papers.txt').read()
#将paper按篇划分为列表
split_paper_list = papers.split('FEDERALIST No. ')

In [3]:
# 存放各个类别paper的列表
H = []
M = []
unknown = []
# 把未知文章的篇数号加到列表中
unknownIndex = []

In [4]:
# 遍历85篇paper查找能用的上的三种分类
# 由于文章编号从1开始因此前面插入一位
for i in range(1, 86):
    paper = split_paper_list[i]
    if "HAMILTON OR MADISON" in paper:
        unknown.append(paper)
        unknownIndex.append(i)
    elif "HAMILTON AND MADISON" in paper:
        pass
    elif "HAMILTON" in paper:
        H.append(paper)
    elif "MADISON" in paper:
        M.append(paper)

# 把他们合成一个列表
papers_list = H + M + unknown

# 统计文章的出现次数超过60词的高频词,组成一个词向量
Vectorizer = CountVectorizer(min_df=60)
#将列表送入Vectorizer统计高频词和次数
feature = Vectorizer.fit_transform(papers_list).toarray()

In [5]:
# 通过MAP计算theta和phi
# 将分好的feature向量切分为三个种类的列表
feature_H = feature[:len(H), :]
feature_M = feature[len(H):len(H) + len(M), :]
feature_Unknown = feature[len(H) + len(M):, :]

In [6]:
# 各关键词关于文章的分布概率
feature_H_sum = feature_H.sum(axis=0)
feature_H_all = feature_H_sum.sum()
feature_H_sum = np.array(feature_H_sum)
theta0 = feature_H_sum / feature_H_all

feature_M_sum = feature_M.sum(axis=0)
feature_M_all = feature_M_sum.sum()
feature_M_sum = np.array(feature_M_sum)
theta1 = feature_M_sum / feature_M_all

In [7]:
# 文章的分布概率
phi0 = len(H) / (len(H) + len(M))

In [8]:
# 将参数做成ndarry方便计算
theta = np.vstack((theta0, theta1))
phi = np.array([phi0, 1-phi0])

def naiveBayes(x_test, theta, phi):
    # 将特征词向量转为01序列
    x_test = np.where(x_test == 0, 0, 1)
    # 计算p(x|y)
    mul = x_test * theta
    #比较概率大小进行判断
    if (np.prod(mul[0]) * phi[0]) > (np.prod(mul[1]) * phi[1]):
        print("Hamilton")
    else:
        print("Madison")

#循环输出最后的预测结果
for index in range(11):
    print("No {} 预测结果为: ".format(unknownIndex[index]), end='')
    predict = naiveBayes(feature_Unknown[index], theta, phi)

No 49 预测结果为: Madison
No 50 预测结果为: Madison
No 51 预测结果为: Madison
No 52 预测结果为: Madison
No 53 预测结果为: Madison
No 54 预测结果为: Madison
No 55 预测结果为: Madison
No 56 预测结果为: Madison
No 57 预测结果为: Madison
No 62 预测结果为: Madison
No 63 预测结果为: Madison
